We will create a churning prediction model for a betting platform. All important information about the problem is in the documentation. <br> 
So, let's begin.

Firstly, let's upload the dataset that we will be using in creating a model.
It is imperative that the files with the given dataset be in your Google Drive -> My Drive folder.

In [1]:
import numpy as np 
import pandas as pd 
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from tables import index
import plotly.express as px 
import matplotlib.pyplot as plt #for visualization
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
#running this cell might take up to 40 seconds because zadatak.csv has 23 million data rows
pd.set_option('display.max_columns', None)
data_df_players = pd.read_csv('/content/drive/My Drive/igraci.csv', index_col = False)
data_df_transactions = pd.read_csv('/content/drive/My Drive/zadatak.csv', sep = ';', index_col = False)

In [34]:
# since transactions dataset has no column names, we assign them to it
data_df_transactions.columns = ['PlayerId', 'TransactionHour', 'Product', 'TransactionType', 'Amount', 'Count']

We can see what our datasets look like, how many features they have and if they have any missing or null values.

In [4]:
def dataoverview(df, message):
    print(f'{message}:')
    print('Number of rows: ', df.shape[0])
    print("Number of features:", df.shape[1])
    print("Data Features:")
    print(df.columns.tolist())
    print("Missing values:", df.isnull().sum().values.sum())
    print("Unique values:")
    print(df.nunique())

In [35]:
dataoverview(data_df_transactions, 'Overview of the transaction dataset')

Overview of the transaction dataset:
Number of rows:  23279661
Number of features: 6
Data Features:
['PlayerId', 'TransactionHour', 'Product', 'TransactionType', 'Amount', 'Count']
Missing values: 2
Unique values:
PlayerId           24465
TransactionHour    17806
Product                7
TransactionType       10
Amount             83264
Count               2975
dtype: int64


In [36]:
# we remove the row that has 2 NaN values and now the data_df_transactions dataset has no more missing values
data_df_transactions.dropna(axis= 0, inplace=True)

In [37]:
dataoverview(data_df_players, 'Overview of the players dataset')

Overview of the players dataset:
Number of rows:  35501
Number of features: 7
Data Features:
['PlayerId', 'Birhtdate', 'City', 'RegistrationDate', 'RegistrationHour', 'IsOptOut', 'RegistrationTerminal']
Missing values: 0
Unique values:
PlayerId                35501
Birhtdate               13973
City                        5
RegistrationDate         2783
RegistrationHour           24
IsOptOut                    2
RegistrationTerminal        6
dtype: int64


Now that we have the datasets, we can start preprocessing the data. Let's convert dates from a string format to a timestamp.

In [38]:
#removing the unnecessary ending 'PM +00:00' that stands for the timezone
data_df_transactions['TransactionHour'] = data_df_transactions['TransactionHour'].str[:-10]
#converting the string format of the date into a timestamp
data_df_transactions['TransactionHour'] = pd.to_datetime(data_df_transactions['TransactionHour'], format="%m/%d/%Y %H:%M:%S")

In [39]:
#changing the typo in the column name in the dataset
data_df_players.rename(columns = {'Birhtdate':'Birthdate'}, inplace = True)

#removing the unnecessary ending 00:00:00.000000 for the time of the day since the date is only important
data_df_players['RegistrationDate'] = data_df_players['RegistrationDate'].str[:-16]
data_df_players['Birthdate'] = data_df_players['Birthdate'].str[:-16]

#converting the string format of the date into a datetime
data_df_players['RegistrationDate'] = pd.to_datetime(data_df_players['RegistrationDate'], format = '%Y-%m-%d')
data_df_players['Birthdate'] = pd.to_datetime(data_df_players['Birthdate'], format = '%Y-%m-%d')

All other data seems to be alright, so there is no more need to preprocess the data. <br>
Let's set a date for when we want our model to be run! Transaction dataset contains information from January 1st 2021 to January 12th 2023 so for our model to work, we need to set a date prior to December 12th 2022. Also, for the model to have enough data to work with, it is best not to set the run date too early(after 1st of May 2021, even though it will work, but not as accurately).

In [66]:
modelRunTimeDay = input("Choose a day of the month for which you want a model to be run! Your input needs be in a number format! ")
modelRunTimeMonth = input("Choose a month for which you want a model to be run! Your input needs be in a number format (1-12)! ")
modelRunTimeYear = input("Choose a year for which you want a model to be run! Your input needs be in a number format (2021-2022)! ")
modelRunTimeDateString = modelRunTimeYear + '-' + modelRunTimeMonth + '-' + modelRunTimeDay
#this is now the date and time when we want our model to run
modelRunTimeDate = datetime.strptime(modelRunTimeDateString, '%Y-%m-%d')
modelRunTimeDate

Choose a day of the month for which you want a model to be run! Your input needs be in a number format! 1
Choose a month for which you want a model to be run! Your input needs be in a number format (1-12)! 5
Choose a year for which you want a model to be run! Your input needs be in a number format (2021-2022)! 2022


datetime.datetime(2022, 5, 1, 0, 0)

Now that we have the date for the model to be run, let's gather all the data that we need. 
First, we need to exclude all those customers who are classified as dormant. A customer is classified as dormant when he/she has not placed any bets or made deposits in the last 30 days. Their inactivity in the past makes them unlikely to start using services again and as such would unbalance the prediction model.

In [43]:
# this is all of the data prior to the set date
relevant_data = data_df_transactions.loc[data_df_transactions['TransactionHour'] <= modelRunTimeDate]

First, let's exclude the dormant customers. Let's get their PlayerId and then remove them from the relevant_data dataframe.

In [44]:
#all the unique playerIds that have transactions up to our model run time date
PlayerIds = relevant_data['PlayerId'].unique()

#let's get playerIds of all players that have been active in the past number_months month
number_months = 1
nondormantPlayerIds = relevant_data['PlayerId'].loc[(relevant_data['TransactionHour'] >= modelRunTimeDate - relativedelta(months=number_months)) &
                                                    ((relevant_data['TransactionType'] == 'Deposit') | (relevant_data['TransactionType'] == 'TicketPayin'))].unique()

# we get dormant players ids by seeing which ids are in PlayerIds.unique() but not in nondormantPlayerIds.unique()
dormantPlayerIds = PlayerIds[~np.isin(PlayerIds,nondormantPlayerIds)]

print(len(PlayerIds))
print(len(nondormantPlayerIds)) # these are the players that we will use in creating our model
print(len(dormantPlayerIds))

# we remove all transactions by those (dormant) players from our relevant_data dataframe
relevant_data.drop(relevant_data[relevant_data['PlayerId'].isin(dormantPlayerIds)].index, inplace=True)

10574
8049
2525


In [ ]:
dataoverview(relevant_data, 'Overview of the transaction dataset before the set run date')

Okay, now we reached a point where we have all the data we need to be able to run our prediction model for the date that we chose. 
Let's set the conditions for labeling our players as churners or not churners.
<br>

In [45]:
# conditions mark that the players with ids next_month_player_id in have made a bet or placed a deposit
# in the following month so they have not churned
condition1 = data_df_transactions['TransactionHour'] >= modelRunTimeDate
condition2 = data_df_transactions['TransactionHour'] <= (modelRunTimeDate + relativedelta(months=1))
condition3 = (data_df_transactions['TransactionType'] == 'Deposit') | (data_df_transactions['TransactionType'] == 'TicketPayin')
next_month_player_id = data_df_transactions.loc[(condition1) & (condition2) & (condition3), 'PlayerId']

We create another dataframe that has all information on our players, and that dataframe will be the one that we train our model on.

In [46]:
# dataframe that only consists of data of our important (nondormant) players
relevant_data_players = data_df_players[data_df_players['PlayerId'].isin(nondormantPlayerIds)]

In [ ]:
dataoverview(relevant_data_players, 'relevant_data_players dataset')

We add the values to the players: 1 if they have churned, 0 if they have not.

In [ ]:
# this cell might take up to 2 minutes to run
# adding values to our players' 'Churn' column
# if a playerId is amongst the next_month_player_ids then they have not churned as we assign the value 0
relevant_data_players['Churn'] = relevant_data_players['PlayerId'].apply(lambda x: 0 if x in next_month_player_id.unique() else 1)

Let's examine how many churners we have.

In [48]:
target_feature = relevant_data_players['Churn'].value_counts().to_frame()
target_feature = target_feature.reset_index()
target_feature = target_feature.rename(columns={'index': 'Category'})
fig = px.pie(target_feature, values='Churn', names='Category', color_discrete_sequence=["green", "red"], title='Distribution of Churn')
fig.show()

Now we have our relevant_data_players dataset with all players who are important to building our model and we have classified them as churners or not churners.
We also have their transaction history in the relevant_data dataset until the date when we run the model. <br> To use both of those datasets at the same time, we will merge them.

In [49]:
merged_df = pd.merge(relevant_data, relevant_data_players, on='PlayerId')

In [ ]:
dataoverview(merged_df, 'Our merged dataset')


We need to get more data for our model to be of high performance. Let's create some features that will be useful for training our model. In the next three cells we are going to create some dataframes that will help us calculate and add those features to our relevant_data_players dataset.

In [50]:
first_transaction_df = merged_df.groupby('PlayerId')['TransactionHour'].min().reset_index()
first_transaction_df = first_transaction_df.rename(columns={'TransactionHour': 'FirstTransactionHour'})
first_transaction_df['TransactionTenure'] = ((modelRunTimeDate.year - pd.to_datetime(first_transaction_df['FirstTransactionHour']).dt.year) * 12 + 
                                            (modelRunTimeDate.month - pd.to_datetime(first_transaction_df['FirstTransactionHour']).dt.month))

The next cell will gather data on the player's transactions in the last month. This will hopefully give us some information on the players latest activities and trends. 

In [51]:
month = merged_df[merged_df['TransactionHour'] >= (modelRunTimeDate - relativedelta(months=1))]

# total value of bonus received by the player in the last month
total_bonus_month_df = month.query('TransactionType == "Bonus"').groupby('PlayerId')['Amount'].sum().reset_index()
total_bonus_month_df = total_bonus_month_df.rename(columns={'Amount': 'MonthBonus'})

# total value of withdrawal made by the player in the last month
total_withdrawal_month_df = month.query('TransactionType == "Withdrawal"').groupby('PlayerId')['Amount'].sum().reset_index()
total_withdrawal_month_df = total_withdrawal_month_df.rename(columns={'Amount': 'MonthWithdrawal'})

# total value of tickets win made by the player in the last month
total_ticketwin_month_df = month.query('TransactionType == "TicketWin"').groupby('PlayerId')['Amount'].sum().reset_index()
total_ticketwin_month_df = total_ticketwin_month_df.rename(columns={'Amount': 'MonthWin'})

# total value of tickets paid by the player in the last month
total_ticketpayin_month_df = month.query('TransactionType == "TicketPayin"').groupby('PlayerId')['Amount'].sum().reset_index()
total_ticketpayin_month_df = total_ticketpayin_month_df.rename(columns={'Amount': 'MonthPayin'})

# total value of deposits by the player in the last month
total_deposit_month_df = month.query('TransactionType == "Deposit"').groupby('PlayerId')['Amount'].sum().reset_index()
total_deposit_month_df = total_deposit_month_df.rename(columns={'Amount': 'MonthDeposit'})

# frequency of deposits made by the player in the last month
count_deposit_month_df = month.query('TransactionType == "Deposit"').groupby('PlayerId')['Count'].count().reset_index()
count_deposit_month_df = count_deposit_month_df.rename(columns={'Count': 'MonthFreqDeposit'})

# frequency of tickets paid by the player in the last month
count_ticketpayin_month_df = month.query('TransactionType == "TicketPayin"').groupby('PlayerId')['Count'].count().reset_index()
count_ticketpayin_month_df = count_ticketpayin_month_df.rename(columns={'Count': 'MonthFreqPayin'})

The next cell gathers data on the player's transactions overall until the date the model is run. We can then compare that to the player's activities in the last month and see changing trends. This will later be useful in training our model.

In [52]:
# total deposits made by the player before the model run date
total_deposit_overall_df = merged_df.query('TransactionType == "Deposit"').groupby('PlayerId')['Amount'].sum().reset_index()
total_deposit_overall_df = total_deposit_overall_df.rename(columns={'Amount': 'TotalDeposit'})

# total ticket payin made by the player before the model run date
total_ticketpayin_overall_df = merged_df.query('TransactionType == "TicketPayin"').groupby('PlayerId')['Amount'].sum().reset_index()
total_ticketpayin_overall_df = total_ticketpayin_overall_df.rename(columns={'Amount': 'TotalPayin'})

# total ticket win made by the player before the model run date
total_ticketwin_overall_df = merged_df.query('TransactionType == "TicketWin"').groupby('PlayerId')['Amount'].sum().reset_index()
total_ticketwin_overall_df = total_ticketwin_overall_df.rename(columns={'Amount': 'TotalWin'})

# frequency of deposits made by the player before the model run date
count_deposit_overall_df = merged_df.query('TransactionType == "Deposit"').groupby('PlayerId')['Count'].count().reset_index()
count_deposit_overall_df = count_deposit_overall_df.rename(columns={'Count': 'TotalDepositCount'})

# frequency of tickets paid by the player before the model run date
count_ticketpayin_overall_df = merged_df.query('TransactionType == "TicketPayin"').groupby('PlayerId')['Count'].count().reset_index()
count_ticketpayin_overall_df = count_ticketpayin_overall_df.rename(columns={'Count': 'TotalPayinCount'})

In [ ]:
# categorizing the product column 
merged_df.loc[merged_df['Product'] == 'Sport', 'Product'] = 1
merged_df.loc[merged_df['Product'] == 'VirtualDogRace', 'Product'] = 2
merged_df.loc[merged_df['Product'] == 'Casino', 'Product'] = 3
merged_df.loc[merged_df['Product'] == 'PaymentProvider', 'Product'] = 4
merged_df.loc[merged_df['Product'] == 'SportTournament', 'Product'] = 5
merged_df.loc[merged_df['Product'] == 'BusinessOwner', 'Product'] = 6
merged_df.loc[merged_df['Product'] == 'VirtualBingo', 'Product'] = 7

# we will give the 'Product' column the value of that product which player used the most
grouped_product_df = merged_df.groupby(['PlayerId', 'Product']).size().reset_index(name='ProductCount')
most_used_products_df = grouped_product_df.groupby('PlayerId')['PlayerId', 'Product', 'ProductCount'].apply(lambda x: x.loc[x['ProductCount'].idxmax()]).reset_index(drop=True)
most_used_products_df.drop('ProductCount', axis = 1, inplace=True)

Let's merge the created datasets to the relevant_data_players dataset.

In [54]:
relevant_data_players = relevant_data_players.merge(first_transaction_df, on='PlayerId')
relevant_data_players = relevant_data_players.merge(total_bonus_month_df, on='PlayerId', how='left').fillna(0)
relevant_data_players = relevant_data_players.merge(total_withdrawal_month_df, on='PlayerId', how='left').fillna(0)
relevant_data_players = relevant_data_players.merge(total_ticketwin_month_df, on='PlayerId', how='left').fillna(0)
relevant_data_players = relevant_data_players.merge(total_ticketpayin_month_df, on='PlayerId', how='left').fillna(0)
relevant_data_players = relevant_data_players.merge(count_deposit_month_df, on='PlayerId', how='left').fillna(0)
relevant_data_players = relevant_data_players.merge(count_ticketpayin_month_df, on='PlayerId', how='left').fillna(0)
relevant_data_players = relevant_data_players.merge(most_used_products_df, on='PlayerId', how='left').fillna(0)
relevant_data_players = relevant_data_players.merge(total_deposit_overall_df, on='PlayerId', how='left').fillna(0)
relevant_data_players = relevant_data_players.merge(total_ticketpayin_overall_df, on='PlayerId', how='left').fillna(0)
relevant_data_players = relevant_data_players.merge(total_ticketwin_overall_df, on='PlayerId', how='left').fillna(0)
relevant_data_players = relevant_data_players.merge(count_deposit_overall_df, on='PlayerId', how='left').fillna(0)
relevant_data_players = relevant_data_players.merge(count_ticketpayin_overall_df, on='PlayerId', how='left').fillna(0)
relevant_data_players = relevant_data_players.merge(total_deposit_month_df, on='PlayerId', how='left').fillna(0)

In [ ]:
dataoverview(relevant_data_players, 'relevant_data_players dataset')

In [55]:
relevant_data_players['MonthRatioWinPayin'] = relevant_data_players['MonthWin'] / relevant_data_players['MonthPayin']
relevant_data_players['OverallRatioWinPayin'] = relevant_data_players['TotalWin'] / relevant_data_players['TotalPayin']
relevant_data_players['OverallAverageDeposit'] = relevant_data_players['TotalDeposit'] / relevant_data_players['TotalDepositCount']
relevant_data_players['OverallAveragePayin'] = relevant_data_players['TotalPayin'] / relevant_data_players['TotalPayinCount']
relevant_data_players['OverallFreqDeposit'] = relevant_data_players['TotalDepositCount'] / relevant_data_players['TransactionTenure']
relevant_data_players['OverallFreqPayin'] = relevant_data_players['TotalPayinCount'] / relevant_data_players['TransactionTenure']
relevant_data_players['Age'] = date.today().year - pd.to_datetime(relevant_data_players['Birthdate']).dt.year
relevant_data_players['RegistrationTenure'] = ((modelRunTimeDate.year - relevant_data_players['RegistrationDate'].dt.year) * 12 + 
                                              (modelRunTimeDate.month - relevant_data_players['RegistrationDate'].dt.month))
relevant_data_players['RatioOverallMonthWinPayin'] = relevant_data_players['MonthRatioWinPayin'] / relevant_data_players['OverallRatioWinPayin']
relevant_data_players['RatioOverallMonthDeposit'] = relevant_data_players['MonthDeposit'] / relevant_data_players['OverallAverageDeposit']

NaN values or infinity values can appear in the cell above for those players who have not made any deposits or placed bets. We will fill those with 0 or 1 respectively.

In [ ]:
dataoverview(relevant_data_players, 'dataset') # we might have some NaN values, and some infinity values (rare but possible)

In [56]:
relevant_data_players.fillna(0, inplace=True)
relevant_data_players = relevant_data_players.replace([np.inf, -np.inf], 1)

Since we used the below features in different ways, we can delete them now. For example, RegistrationDate was used to create RegistrationTenure, feature that shows the number of monhts since the registration date by the player.

In [57]:
# we don't need these columns so we can delete them
relevant_data_players.drop(['Birthdate', 'RegistrationHour', 'FirstTransactionHour', 'RegistrationDate'], axis = 1, inplace=True)

Let's categorize and give numerical values to our features so the model can train on them.

In [58]:
# categorizing the Age column
relevant_data_players.loc[relevant_data_players['Age'] < 40, 'Age'] = 1
relevant_data_players.loc[(relevant_data_players['Age'] >= 40) & (relevant_data_players['Age'] <= 65), 'Age'] = 2
relevant_data_players.loc[relevant_data_players['Age'] > 65, 'Age'] = 3

# giving a numerical value to the RegistrationTerminal column
relevant_data_players.loc[relevant_data_players['RegistrationTerminal'] == "Web", 'RegistrationTerminal'] = 0
relevant_data_players.loc[relevant_data_players['RegistrationTerminal'] == "MobileWeb", 'RegistrationTerminal'] = 1
relevant_data_players.loc[relevant_data_players['RegistrationTerminal'] == "Mobile Web", 'RegistrationTerminal'] = 1
relevant_data_players.loc[relevant_data_players['RegistrationTerminal'] == "Android", 'RegistrationTerminal'] = 2
relevant_data_players.loc[relevant_data_players['RegistrationTerminal'] == "iOS", 'RegistrationTerminal'] = 3
relevant_data_players.loc[relevant_data_players['RegistrationTerminal'] == "Desktop Web", 'RegistrationTerminal'] = 4

# giving a numerical value to the City column
relevant_data_players.loc[relevant_data_players['City'] == "Trogir", 'City'] = 0
relevant_data_players.loc[relevant_data_players['City'] == "Rijeka", 'City'] = 1
relevant_data_players.loc[relevant_data_players['City'] == "Pazin", 'City'] = 2
relevant_data_players.loc[relevant_data_players['City'] == "Cres", 'City'] = 3
relevant_data_players.loc[relevant_data_players['City'] == "Dubrovnik", 'City'] = 4

# giving a numerical value to the IsOptOut column
relevant_data_players.loc[relevant_data_players['IsOptOut'] == False, 'IsOptOut'] = 0
relevant_data_players.loc[relevant_data_players['IsOptOut'] == True, 'IsOptOut'] = 1

Now we have all of our data gathered in one dataset relevant_data_players and from the features of this dataset we will build a model predicting the value of a Churn feature. <br>

Let's see how our features correlate with one another. Those who have high correlation between themselves are more linearly dependent, so we will be able to drop one of them when creating our model since they have the same effect on our target variable (for example, TotalWin and TotalPayin are highly correlated).

In [59]:
corr = relevant_data_players.corr()

fig = px.imshow(corr,width=700, height=700)
fig.show()

Now, let’s use the generalized linear model to gain some statistics of the respective features with the target.

In [60]:
glm_columns = [c for c in relevant_data_players.columns if c not in ['customerID', 'Churn']]
glm_columns = ' + '.join(map(str, glm_columns))

glm_model = smf.glm(formula=f'Churn ~ {glm_columns}', data=relevant_data_players, family=sm.families.Binomial())
res = glm_model.fit()
print(res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Churn   No. Observations:                 8049
Model:                            GLM   Df Residuals:                     8014
Model Family:                Binomial   Df Model:                           34
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2922.9
Date:                Fri, 03 Mar 2023   Deviance:                       5845.7
Time:                        21:32:34   Pearson chi2:                 1.86e+06
No. Iterations:                    10   Pseudo R-squ. (CS):             0.2096
Covariance Type:            nonrobust                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

P>|z| column tells us which features make customers churn or not. If the value of the column is less than 0.05, it means that the feature affects Churn in a statistically significant way. <br>
The second fact here is which features are most important for training our model with high performance. That we can find out by looking at the exponential coefficient values, which estimate the expected change in churn through a given feature by a change of one unit. <br>
Values more than 1 indicate increased churn, while values less than 1 indicate that churn is happening less.

In [61]:
np.exp(res.params)

Intercept                    2.890057
City[T.1]                    1.088871
City[T.2]                    1.141694
City[T.3]                    1.206654
City[T.4]                    1.100841
IsOptOut[T.1]                1.224180
RegistrationTerminal[T.1]    0.911405
RegistrationTerminal[T.2]    0.799143
RegistrationTerminal[T.3]    0.919693
RegistrationTerminal[T.4]    0.686912
PlayerId                     1.000019
TransactionTenure            0.612706
MonthBonus                   1.002275
MonthWithdrawal              1.001274
MonthWin                     0.998769
MonthPayin                   1.001267
MonthFreqDeposit             0.875394
MonthFreqPayin               0.990747
Product                      1.020191
TotalDeposit                 1.000339
TotalPayin                   0.999588
TotalWin                     1.000369
TotalDepositCount            1.026101
TotalPayinCount              1.005289
MonthDeposit                 0.997820
MonthRatioWinPayin           1.134504
OverallRatio

The range of all features should be normalized so that each feature contributes approximately proportionately to the final distance, so we do feature scaling.

In [62]:
sc = MinMaxScaler()
relevant_data_players['RegistrationTenure'] = sc.fit_transform(relevant_data_players[['RegistrationTenure']])
relevant_data_players['TransactionTenure'] = sc.fit_transform(relevant_data_players[['TransactionTenure']])
relevant_data_players['MonthBonus'] = sc.fit_transform(relevant_data_players[['RegistrationTenure']])
relevant_data_players['MonthWithdrawal'] = sc.fit_transform(relevant_data_players[['MonthWithdrawal']])
relevant_data_players['MonthWin'] = sc.fit_transform(relevant_data_players[['MonthWin']])
relevant_data_players['MonthPayin'] = sc.fit_transform(relevant_data_players[['MonthPayin']])
relevant_data_players['MonthFreqDeposit'] = sc.fit_transform(relevant_data_players[['MonthFreqDeposit']])
relevant_data_players['MonthFreqPayin'] = sc.fit_transform(relevant_data_players[['MonthFreqPayin']])
relevant_data_players['TotalDeposit'] = sc.fit_transform(relevant_data_players[['TotalDeposit']])
relevant_data_players['TotalPayin'] = sc.fit_transform(relevant_data_players[['TotalPayin']])
relevant_data_players['TotalWin'] = sc.fit_transform(relevant_data_players[['TotalWin']])
relevant_data_players['TotalDepositCount'] = sc.fit_transform(relevant_data_players[['TotalDepositCount']])
relevant_data_players['TotalPayinCount'] = sc.fit_transform(relevant_data_players[['TotalPayinCount']]) 
relevant_data_players['OverallAverageDeposit'] = sc.fit_transform(relevant_data_players[['OverallAverageDeposit']])
relevant_data_players['OverallAveragePayin'] = sc.fit_transform(relevant_data_players[['OverallAveragePayin']])
relevant_data_players['OverallFreqDeposit'] = sc.fit_transform(relevant_data_players[['OverallFreqDeposit']])
relevant_data_players['OverallFreqPayin'] = sc.fit_transform(relevant_data_players[['OverallFreqPayin']])
relevant_data_players['MonthRatioWinPayin'] = sc.fit_transform(relevant_data_players[['MonthRatioWinPayin']])
relevant_data_players['OverallRatioWinPayin'] = sc.fit_transform(relevant_data_players[['OverallRatioWinPayin']])
relevant_data_players['RatioOverallMonthWinPayin'] = sc.fit_transform(relevant_data_players[['RatioOverallMonthWinPayin']])
relevant_data_players['RatioOverallMonthDeposit'] = sc.fit_transform(relevant_data_players[['RatioOverallMonthDeposit']])

Let’s start creating a baseline model with a Logistic Regression algorithm. 

In [63]:
from sklearn.linear_model import LogisticRegression 
class CustomLogisticRegression(LogisticRegression):   # we use the custom model so that we can pass both parameters max_iter and class_weight
    
    def __init__(self, solver='lbfgs', max_iter=1000, class_weight='balanced', **kwargs):
        super().__init__(solver=solver, max_iter=max_iter, class_weight=class_weight, **kwargs)

In [ ]:
# import metric for performance evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# split data into train and test sets
from sklearn.model_selection import train_test_split

X = relevant_data_players.drop(['Churn', 'PlayerId', 'MonthDeposit', 'OverallAveragePayin', 'City'], axis=1)
y = relevant_data_players['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

def modeling(alg, alg_name, params={}):
    model = alg() 
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
     
    tp = fp = tn = fn = 0 
    for (i, j) in zip(y_test, y_pred):
      if j == 0:
        if i == 0:
          tn = tn + 1
        else:
          fn = fn + 1
      else:
        if i == 0:
          fp = fp + 1
        else:
          tp = tp + 1
    print('tp, tn, fp, fn: ' + str(tp) + ' ' + str(tn) + ' ' + str(fp) + ' ' + str(fn))
  
    #Performance evaluation
    def print_scores(alg, y_true, y_pred):
        print(alg_name)
        acc_score = accuracy_score(y_true, y_pred)
        print("accuracy: ",acc_score)
        pre_score = precision_score(y_true, y_pred)
        print("precision: ",pre_score)
        rec_score = recall_score(y_true, y_pred)
        print("recall: ",rec_score)
        f_score = f1_score(y_true, y_pred, average='weighted')
        print("f1_score: ",f_score)

    print_scores(alg, y_test, y_pred)
    return model

# Running logistic regression model
log_model = modeling(CustomLogisticRegression, 'Logistic Regression')

We can see that accuracy is not great, but what is important in building a churnig prediction model is to have a good recall, that is not to have a lot of false negatives, which are churners that were labeled as non-churners. Let's see if other models will be more successful.

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# Running SVC model
log_model = modeling(SVC, 'Support Vector Classifier')

# Running RandomForestClassifier model
log_model = modeling(RandomForestClassifier, 'Random Forest Classifier')

# Running DecisionTreeClassifier model
log_model = modeling(DecisionTreeClassifier, 'Decision Tree Classifier')

# Running GaussianNB model
log_model = modeling(GaussianNB, 'Gaussian Naive Bayes')

We can see that Random forest classifier is higher in accuracy, but its recall is low, which is not good.
Other models are not so accurate given our features, except Naive Bayes which has a good level of recall. <br>
To better improve upon the models, we would need to collect more data or engineer our features in a way that they would be more useful in training our model.

References: https://neptune.ai/blog/how-to-implement-customer-churn-prediction#:~:text=Importance%20of%20customer%20churn%20prediction&text=Machine%20learning%20and%20data%20analysis,churn%20and%20increase%20customer%20retention. <br>
https://www.kdnuggets.com/2022/07/logistic-regression-work.html#:~:text=is%20Logistic%20Regression%3F-,Logistic%20regression%20is%20a%20Machine%20Learning%20classification%20algorithm%20that%20is,the%20logistic%20of%20the%20result. <br>
https://stackoverflow.com/ <br>
chatGPT helping with python syntax and pandas dataframe management

Author: Fran Zvonimir Zgombić